### 데이터 분석 연습하기
1. 고객 3500명에 대한 학습용 데이터를 이용하여 성별 예측 모형을 만든 후 <br>
이를 평가용 데이터에 적용하여 얻은 2482명 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하라<br>
제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

In [1]:
#라이브러리와 파일 읽기
import pandas as pd
x_train = pd.read_csv('x_train.csv', encoding = 'CP949')
x_test = pd.read_csv('x_test.csv', encoding = 'CP949')
y_train = pd.read_csv('y_train.csv', encoding = 'CP949')

In [2]:
# 테스트 데이터의 cust_id 저장하기
x_test_cust_id = x_test['cust_id']

In [3]:
#cust_id 칼럼 삭제하기
x_train=x_train.drop(columns=['cust_id'])
x_test=x_test.drop(columns=['cust_id'])
y_train=y_train.drop(columns=['cust_id'])

In [4]:
# 결측치 처리하기
x_train['환불금액']=x_train['환불금액'].fillna(0)
x_test['환불금액']=x_test['환불금액'].fillna(0)

In [5]:
# 라벨 인코딩 수행하기
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
x_train['주구매상품']=encoder.fit_transform(x_train['주구매상품'])
x_test['주구매상품']=encoder.fit_transform(x_test['주구매상품'])
x_train['주구매지점']=encoder.fit_transform(x_train['주구매지점'])
x_test['주구매지점']=encoder.fit_transform(x_test['주구매지점'])

In [6]:
#파생변수 만들기
condition = x_train['환불금액']>0
x_train.loc[condition,'환불금액_new']=1
x_train.loc[~condition,'환불금액_new']=0
x_train=x_train.drop(columns=['환불금액'])
x_test.loc[condition,'환불금액_new']=1
x_test.loc[~condition,'환불금액_new']=0
x_test=x_test.drop(columns=['환불금액'])

In [7]:
# 데이터 스케일링 : 표준화 크기로 변환하기
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train),columns=x_train.columns)
x_test=pd.DataFrame(scaler.fit_transform(x_test),columns= x_test.columns)

In [8]:
# 불필요한 칼럼 삭제하기
x_train = x_train.drop(columns=['최대구매액'])
x_test = x_test.drop(columns=['최대구매액'])

In [9]:
# 학습용/검증용 데이터로 분리하기
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train,y_train,\
                                                   test_size=0.2, random_state=10)

In [13]:
# 모델 학습 및 평가하기
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=10 , criterion='entropy',
                              random_state=10)
model.fit(X_TRAIN,Y_TRAIN)
y_test_proba= model.predict_proba(X_TEST)

In [14]:
#결과 제출하기
result = pd.DataFrame(y_test_proba)[1]
final = pd.concat([x_test_cust_id,result], axis=1).rename(columns={1:'gender'})
final.to_csv('12345.csv',index=False)